# The First Example
Usually you start off with a "hello world" example.  But "hello world" would be too boring in F# so let us start with something a little more interesting and fun.  In this first example we will create a dataset (i.e. an array) that we will show in a histogram.

In [26]:
type PersonPolicy = 
    {
        PersonId: string;
        PolicyNumber: string;
        Premium: decimal;
    }
    
let data = 
    [| 
        { PersonId = "P1"; PolicyNumber = "Pol01"; Premium = 100m };
        { PersonId = "P1"; PolicyNumber = "Pol02"; Premium = 200m };
        { PersonId = "P2"; PolicyNumber = "Pol03"; Premium = 150m };
        { PersonId = "P3"; PolicyNumber = "Pol04"; Premium = 250m };
        { PersonId = "P3"; PolicyNumber = "Pol05"; Premium = 350m };
    |]
    
data

index PersonId PolicyNumber Premium 0 P1 Pol01 100 1 P1 Pol02 200 2 P2 Pol03 150 3 P3 Pol04 250 4 P3 Pol05 350

The `PersonPolicy` type describes how each row in our dataset will look like.  Each row has a person ID, a policy number and a premium.  This is similar to columns in a table as shown by the `display` command.  The value `data` is an array of persons where each person can have one or more policies.

Let us draw a diagram.  We want a bar diagram with the policy number on one axis and the premium on the other.  That means we want to create data for the two axis from our existing `data` values.  F# has a built-in function for creating data or "mapping" data called `map`.  So we will use `Array.map` to create data for the diagram.

In [27]:
let graphData = data |> Array.map (fun row -> row.PolicyNumber, row.Premium)

graphData

index Item1 Item2 0 Pol01 100 1 Pol02 200 2 Pol03 150 3 Pol04 250 4 Pol05 350

Note the use of the "pipe" operator `|>`.  The pipe operator "pipes" data into a function and it is extremely important and useful when working with data in F#.  We will get back to it later in this tutorial.  The part after the "fun" keyword is a function that tells `Array.map` that we want the policy number and the premium from each row.

We are now ready to create the plot.

In [19]:
graphData
|> Chart.Bar
|> Chart.WithLabels ["Policies"; "Premiums"]

That looks nice but what we really want is to group the policies and show the total premium for each person.

Let us create a type that holds the premium per person.

In [20]:
type PersonPremium = 
    {
        PersonId: string;
        Premium: decimal;
    }

We also need a function that sums the premiums for an array of `PersonPolicy`.  We will use the function `Array.sumBy` to do that.

In [21]:
let sumPremium (personPolicies: PersonPolicy[]) =
    personPolicies |> Array.sumBy (fun personPolicy -> personPolicy.Premium)

We can test it by summing the premiums for our entire dataset.

In [22]:
sumPremium data

1050

That looks just about right, except we do not want to get the total premium.  Instead we want the premium for each person so first we need to group the dataset per person and then sum the premiums.  Fortunately, F# has a built-in function for grouping arrays: `Array.groupBy`.  After grouping we use `Array.map` to transform the grouped data into an array of `PersonPremium` elements and we apply our function `sumPremium` to sum the premiums for each person.

In [25]:
let groupedByPerson = 
    data
    |> Array.groupBy (fun personPolicy -> personPolicy.PersonId)
    |> Array.map (fun (personId, personPolicies) -> 
        {
            PersonId = personId; 
            Premium = sumPremium personPolicies 
        })
    
groupedByPerson


index PersonId Premium 0 P1 300 1 P2 150 2 P3 600

Pretty neat! Let's plot it.

In [24]:
groupedByPerson
|> Array.map (fun g -> g.PersonId, g.Premium)
|> Chart.Bar
|> Chart.WithLabels ["Person"; "Premiums"]

## Conclusion
This example hopefully gave you an idea of the power of F#.  It showed you how you can create datasets as arrays and how you can transform and plot data.  Don't worry if you did not understand all of it.  I would be surprised if you did.  We will dive deeper into the concepts of data and functions in the rest of the guide.
